In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.constants
import math
import re
from isq import QuantityValue

In [3]:
def getScipyConstant(key):
    return QuantityValue(scipy.constants.value(key),
                  scipy.constants.unit(key),
                  scipy.constants.precision(key) * abs(scipy.constants.value(key)))

In [4]:
pi = scipy.constants.pi
G = getScipyConstant('Newtonian constant of gravitation')
c = getScipyConstant('speed of light in vacuum')
g = getScipyConstant('standard acceleration of gravity')
au = QuantityValue(149597870700, 'm', 0)
mu_0 = getScipyConstant('mag. constant')

- Mass flow past skyhook, $mV_O$
- Velocity change at skyhook, $V_O\Delta\theta$
- Rate of change of momentum, $mV^2_O\Delta\theta$
- Resultant force of tension, $F_T = mV^2_O\Delta\theta$

$P = \exp(\frac{-\rho g H}{Y})$

$A_T = A_0 \exp(\frac{\rho g}{Y} \cdot \frac{RH}{R+H})$

In [5]:
def areaTop(payload, density, tensileStrength, altitude):
    return payload * g / tensileStrength * (density * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [6]:
def forceTop(payload, density, tensileStrength, altitude):
    return payload * g * (density * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [7]:
linearDensity = QuantityValue(.01, 'kg m^-1', 0)
angle = QuantityValue(0.25, '1', 0)
altitude = QuantityValue(200e3, 'm', 5e3)
R_Earth = QuantityValue(6378137, 'm', 2) #WGS84
M_Earth = QuantityValue(5.972e24, 'kg', 0)
M_Sol = QuantityValue(1.98855e30, 'kg', 0.00025e30)
payload = QuantityValue(2e3, 'kg', 0)
tensileStrength = QuantityValue(3E9, 'kg m s^-2 m^-2', 0)
massFlowRate = 1 # kg/s
massDepositionEfficiency = .5
speed = (G * M_Earth / (R_Earth + altitude)).sqrt()
ladderDensity = QuantityValue(2e3, 'kg m^-3', 500)
hookMass = QuantityValue(5e3, 'kg', 2.5e3)
P = (-ladderDensity * g / tensileStrength * altitude).exp()
ladderMass = payload / P
ringMass = ((R_Earth + altitude) * linearDensity * 2 * math.pi)
hookLift = linearDensity * speed * speed * angle
numHooks = 2
totalMass = (ringMass + (numHooks * hookMass) + (numHooks * ladderMass))
numLaunches = totalMass / QuantityValue(63800, 'kg launches^-1', 0)
force = forceTop(payload, ladderDensity, tensileStrength, altitude)
hookLoad = force + hookMass * g * R_Earth*R_Earth/(R_Earth+altitude)/(R_Earth+altitude)
area = areaTop(payload, ladderDensity, tensileStrength, altitude)
B = QuantityValue(3.5, 'kg A^-1 s^-2', 0)
I = QuantityValue(100000, 'A', 0)

In [8]:
print('Altitude: {}\nRing speed: {}\nHook capacity: {}\nHook load: {}\nDesign payload: {}\n\nRing mass: {}\nHook mass: {}\nLadder mass: {}\nTotal mass: {}\n\nLaunches: {}'.format(
    altitude, speed, hookLift, hookLoad, payload, ringMass, hookMass, ladderMass, totalMass, numLaunches))

Altitude: 200.0(50)e3 m
Ring speed: 7.7840(30)e3 m s^-1
Hook capacity: 151.478(82)e3 kg s^-2 m
Hook load: 116(32)e3 kg m s^-2
Design payload: 2.0000(...)e3 kg

Ring mass: 413.32(31)e3 kg
Hook mass: 5.0(25)e3 kg
Ladder mass: 7.4(24)e3 kg
Total mass: 438.1(70)e3 kg

Launches: 6.87(11) launches
